In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt

In [3]:
dataset_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/'
training_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/train/'
validation_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/val/'
test_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/test/'
model_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/vgg16_model.h5'

IMAGE_SIZE = [224, 224,3]
EPOCHS = 30
TARGET_SIZE = (224,224)
BATCH_SIZE = 16

In [4]:
# useful for getting number of output classes
folders = glob(training_path+'*')
print(len(folders))

4


In [5]:
# importing the resnet and using the imagenet weights
# Here we will be using imagenet weights

vgg_16_model = VGG16(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE)

58892288/58889256 [==============================] - 1s 0us/step


In [6]:
# don't train existing weights
for layer in vgg_16_model.layers:
  layer.trainable = False

In [7]:
# our layers - you can add more if you want
x_network = Flatten()(vgg_16_model.output)
# adding the prediction layer
prediction_layer = Dense(len(folders),activation='softmax')(x_network)

In [8]:
# create a model object
model = Model(inputs = vgg_16_model.input , outputs = prediction_layer)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [9]:
# compiling the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
# creating callbacks to save best model
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath= model_path,
                                                               save_weights_only=False,
                                                               monitor='val_accuracy',
                                                               save_best_only=True)


In [11]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs = None):
    k = list(logs.keys())
    # print(k)
    if(logs.get('val_accuracy') > 0.99):
      print('Required accuracy reached , so stopping training.....!!')
      self.model.stop_training = True
  

In [12]:
# creating datagenerators

# train datagenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# test datagenerator
test_datagen = ImageDataGenerator(rescale= 1./255)

In [13]:
training_data = train_datagen.flow_from_directory(training_path,
                                                  target_size = TARGET_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode = 'categorical')

validation_dataset = test_datagen.flow_from_directory(validation_path,
                                                   target_size = TARGET_SIZE,
                                                   batch_size = BATCH_SIZE,
                                                   class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.


In [14]:
# fit the model

r = model.fit_generator(training_data,
                        validation_data = validation_dataset,
                        epochs = EPOCHS,
                        steps_per_epoch = len(training_data),
                        validation_steps = len(validation_dataset),
                        callbacks=[model_checkpoint_callback , MyCallback()])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
122/122 [==============================] - 1356s 11s/step - loss: 0.5054 - accuracy: 0.8037 - val_loss: 0.3070 - val_accuracy: 0.8981
Epoch 2/30
122/122 [==============================] - 37s 300ms/step - loss: 0.2069 - accuracy: 0.9185 - val_loss: 0.2870 - val_accuracy: 0.8858
Epoch 3/30
122/122 [==============================] - 40s 329ms/step - loss: 0.1747 - accuracy: 0.9349 - val_loss: 0.1296 - val_accuracy: 0.9352
Epoch 4/30
122/122 [==============================] - 37s 301ms/step - loss: 0.1332 - accuracy: 0.9482 - val_loss: 0.2303 - val_accuracy: 0.9105
Epoch 5/30
122/122 [==============================] - 40s 332ms/step - loss: 0.1352 - accuracy: 0.9534 - val_loss: 0.2066 - val_accuracy: 0.9383
Epoch 6/30
122/122 [==============================] - 36s 298ms/step - loss: 0.0915 - accuracy: 0.9662 - val_loss: 0.2442 - val_accuracy: 0.9167
Epoch 7/30
122/122 [==============================] - 